<a href="https://colab.research.google.com/github/bwsi-hadr/2019-student-final-exercise/blob/master/logistics_output_files_for_turns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!export CURL_CA_BUNDLE=/etc/ssl/certs/ca-certificates.crt
!sudo mkdir -p /etc/pki/tls/certs
!sudo cp /etc/ssl/certs/ca-certificates.crt /etc/pki/tls/certs/ca-bundle.crt
try:
  import rasterio
  import rasterio.plot
  import rasterio.merge 
  import rasterio.mask
except:
  !pip install rasterio
  import rasterio
  import rasterio.plot
  import rasterio.merge
  import rasterio.mask
  
try:
  import rasterstats as rs
except:
  !pip install rasterstats  
  import rasterstats as rs
  
try:
  import pyproj
except:
  !pip install pyproj
  import pyproj
  
import networkx as nx
try:
  import osmnx as ox
except:
  # osmnx depends on the system package libspatialindex
  !apt install libspatialindex-dev
  !pip install osmnx
  import osmnx as ox

try: 
  import geopandas as gpd
except: 
  !pip install geopandas 
  import geopandas as gpd
  
try:
  import contextily as ctx 
except:
  # install dependencies for contextily
  !apt install libproj-dev proj-data proj-bin
  !apt install libgeos-dev
  !pip install cython
  !pip install cartopy
  # install contextily
  !pip install contextily==1.0rc1 --no-use-pep517 --no-cache-dir
  import contextily as ctx
  
import fiona
from shapely.geometry import Point, LineString, Polygon
  
import gdal
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import pathlib
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/'My Drive'/BWSI-Remote-Sensing/'Final_exercise'/'game2'/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/BWSI-Remote-Sensing/Final_exercise/game2


In [0]:
global add_supply
def reset_supply():
  global add_supply
  add_supply = pd.DataFrame(columns=['objectid','type','food_water','medicine','fuel'])
def init_supply():
  global add_supply
  add_supply.loc[0,'objectid']=0
  add_supply.fillna(0)
def unpack_powerplants(locations):
  global add_supply
  for i,idx in enumerate(locations):
    add_supply.loc[idx, 'type']='power_plant'
    add_supply.loc[idx, 'food_water']=0
    add_supply.loc[idx, 'medicine']=0
    add_supply.loc[idx, 'fuel']=300
    add_supply.loc[idx, 'objectid']=idx
def unpack_airfields(locations):
  global add_supply
  for i,idx in enumerate(locations):
    add_supply.loc[idx, 'type']='airfield'
    add_supply.loc[idx, 'food_water']=0
    add_supply.loc[idx, 'medicine']=0
    add_supply.loc[idx, 'fuel']=90
    add_supply.loc[idx, 'objectid']=idx
def add_resources(destind, desttype, foodwater, medicine, fuel):
  global add_supply
  add_supply.loc[destind, 'type'] = desttype
  add_supply = add_supply.fillna(0)
  add_supply.loc[destind, 'food_water'] += foodwater
  add_supply.loc[destind, 'medicine'] += medicine
  add_supply.loc[destind, 'fuel'] += fuel
  add_supply.loc[destind, 'objectid'] = destind
def unpack_large_hospitals(locations):
  global add_supply
  for i,idx in enumerate(locations):
    add_supply.loc[idx, 'type']='large_hospital'
    add_supply.loc[idx, 'food_water']=300
    add_supply.loc[idx, 'medicine']=300
    add_supply.loc[idx, 'fuel']=0
    add_supply.loc[idx, 'objectid']=idx
def unpack_small_hospitals(locations):
  global add_supply
  for i,idx in enumerate(locations):
    add_supply.loc[idx, 'type']='small_hospital'
    add_supply.loc[idx, 'food_water']=150
    add_supply.loc[idx, 'medicine']=150
    add_supply.loc[idx, 'fuel']=0
    add_supply.loc[idx, 'objectid']=idx
def unpack_shelters(locations):
  global add_supply
  for i,idx in enumerate(locations):
    #filew.writerow([idx,'airfield',0,0,300])
    add_supply.loc[idx, 'type']='shelter'
    add_supply.loc[idx, 'food_water']=100
    add_supply.loc[idx, 'medicine']=0
    add_supply.loc[idx, 'fuel']=0
    add_supply.loc[idx, 'objectid']=idx
def unpack_cell_tower(locations):
  global add_supply
  for i,idx in enumerate(locations):
    #filew.writerow([idx,'airfield',0,0,300])
    add_supply.loc[idx, 'type']='shelter'
    add_supply.loc[idx, 'food_water']=0
    add_supply.loc[idx, 'medicine']=0
    add_supply.loc[idx, 'fuel']=0
    add_supply.loc[idx, 'objectid']=idx
def distribute_supply(locations,types,foodwater,medicine,fuel):
  global add_suppy
  for i,ix in enumerate(locatins):
    add_supply.loc[idx, 'type'] = types
    add_supply.loc[idx, 'food_water']=foodwater
    add_supply.loc[idx, 'medicine']=medicine
    add_supply.loc[idx, 'fuel']=fuel
    add_supply.loc[idx, 'objectid']=idx
def finalize(exist, path='resources.csv'):
  global add_supply
  if exist:
    existing = pd.DataFrame.from_csv(path)
    new = existing.append(add_supply).fillna(0)
    new.to_csv(path)
    print(new)
  else:
    add_supply.copy().to_csv(path)

In [0]:
global deploy_asset
def reset_deploy():
  global deploy_asset
  deploy_asset = gpd.GeoDataFrame(columns=['objectid','type','geometry'])
  deploy_asset.crs={'init': 'epsg:3785', 'no_defs': True}
def deploy(assetid,assettype,geometry):
  global deploy_asset
  deploy_asset.loc[assetid, 'objectid']=assetid
  deploy_asset.loc[assetid, 'type']=assettype
  deploy_asset.loc[assetid, 'geometry']=geometry
def deploy_distribute(assetidstart,assettype,geometries):
  counter = assetidstart
  for i in geometries:
    deploy_asset.loc[counter,'objectid']=counter
    deploy_asset.loc[counter, 'type']=assettype
    deploy_asset.loc[counter, 'geometry']=i
    counter += 1
def finalize_assets(exist,path='assets.shp'):
  global deploy_asset
  if exist:
    existing = gpd.GeoDataFrame.from_file(path)
    new = existing.append(deploy_asset).fillna(0)
    print(new)
    new.to_file(path)
  else:
    deploy_asset.copy().to_file(path)

Turn 0:

In [0]:
reset_supply()
unpack_powerplants([338.0, 564.0, 397.0, 233.0, 642.0, 466.0, 21.0, 481.0, 512.0, 546.0, 486.0, 658.0, 459.0, 460.0, 577.0, 180.0, 59.0, 39.0, 219.0, 526.0])

In [0]:
unpack_airfields([7519, 7661, 7690, 7650, 7695])

In [0]:
add_resources(7695,'airfield',150000,150000,37500)

In [0]:
add_supply #check order

,objectid,type,food_water,medicine,fuel
338.0,338.0,power_plant,0,0,300
564.0,564.0,power_plant,0,0,300
397.0,397.0,power_plant,0,0,300
233.0,233.0,power_plant,0,0,300
642.0,642.0,power_plant,0,0,300
466.0,466.0,power_plant,0,0,300
21.0,21.0,power_plant,0,0,300
481.0,481.0,power_plant,0,0,300
512.0,512.0,power_plant,0,0,300
546.0,546.0,power_plant,0,0,300


In [0]:
finalize(False) #False if file doesn't already exist